# Advanced Search - Chaining
---

Before we do any work, we need to import several functions from cdapython:


- `Q` and `query` which power the search
- `columns` which lets us view entity field names
- `unique_terms` which lets view entity field contents
    
We're also asking cdapython to report it's version so we can be sure we're using the one we mean to.

In [1]:
from cdapython import Q, columns, unique_terms, query
import cdapython
print(cdapython.__version__)
Q.set_host_url("http://35.192.60.10:8080/")

2022.5.9



The CDA provides a custom python tool for searching CDA data. [`Q`](usage/#q) (short for Query) offers several ways to search and filter data, and several input modes:

---

- **[Q.run()](../../../Documentation/usage/#qrun)** returns data for the specified search 
- **[Q.counts()](../../../Documentation/usage/#qcounts)** returns summary information (counts) for the files that fit the specified search
- **[Q.sql()](../../../Documentation/usage/#qsql)** allows you to use SQL syntax instead of Q syntax 
- **[query()](../../../Documentation/usage/#query)** allows you to use a more natural language syntax of Q

---

## Chaining

We're going to build on our [previous search](../BasicSearch) to see what information exists about cancers that were first diagnosed in the brain. 


In [2]:
myquery = Q('ResearchSubject.primary_diagnosis_site = "brain"')


Previously we looked at subject, research_subject, specimen and file results seperately, but a more interesting search is to combine these. Let's say what we're really interested in is finding analysis done on specimens, so we're looking for files that belong to specimens that match our search. For that, we want to chain our query to the specimen endpoint and then to the files endpoint and get the combined result:

In [3]:
myqueryspecimenfiles =  myquery.specimen.files.run()
myqueryspecimenfiles

Getting results from database

Total execution time: 3622 ms



            QueryID: 3845d4c4-0e99-490c-b2f3-fa59288df875
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Files_v3_0_w_RS.id) as rn, all_Files_v3_0_w_RS.id AS id, all_Files_v3_0_w_RS.identifier AS identifier, all_Files_v3_0_w_RS.label AS label, all_Files_v3_0_w_RS.data_category AS data_category, all_Files_v3_0_w_RS.data_type AS data_type, all_Files_v3_0_w_RS.file_format AS file_format, all_Files_v3_0_w_RS.associated_project AS associated_project, all_Files_v3_0_w_RS.drs_uri AS drs_uri, all_Files_v3_0_w_RS.byte_size AS byte_size, all_Files_v3_0_w_RS.checksum AS checksum, all_Files_v3_0_w_RS.data_modality AS data_modality, all_Files_v3_0_w_RS.imaging_modality AS imaging_modality, all_Files_v3_0_w_RS.dbgap_accession_number AS dbgap_accession_number FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS LEFT JOIN UNNEST(all_Subjects_v3_0_w_RS.ResearchSubject) AS _ResearchSubject LEFT JOIN UNNEST(_ResearchSubject.Specimen) AS 

In [4]:
myqueryspecimenfiles.to_dataframe()

,id,identifier,label,data_category,data_type,file_format,associated_project,drs_uri,byte_size,checksum,data_modality,imaging_modality,dbgap_accession_number
0,1676bc16-a120-4951-bcb9-b3e0599b45af,"[{'system': 'GDC', 'value': '1676bc16-a120-495...",7dee3b3f-7f68-493b-a99c-49ae65675609.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-MSK,drs://dg.4DFC:1676bc16-a120-4951-bcb9-b3e0599b...,1051,ecabcaa16c4f79262c996b5df904d3c3,Genomic,None,None
1,1825d954-56e3-4faa-991e-56e28d23f31e,"[{'system': 'GDC', 'value': '1825d954-56e3-4fa...",415b62cf-1ab3-4432-9fd4-c326718d8142.genie.ali...,Copy Number Variation,Gene Level Copy Number Scores,TSV,GENIE-MSK,drs://dg.4DFC:1825d954-56e3-4faa-991e-56e28d23...,22932,78fb97b022c9790c89888ba62bfb222e,Genomic,None,None
2,3c4c6124-e58f-497a-8a20-e5ade9caa765,"[{'system': 'GDC', 'value': '3c4c6124-e58f-497...",c7a985b2-bc10-4e58-86e8-0ef41f41f80c.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-JHU,drs://dg.4DFC:3c4c6124-e58f-497a-8a20-e5ade9ca...,1053,d7dabf1852034e3fc333ae68b38f2281,Genomic,None,None
3,9148502d-cb91-41c6-b382-62639010ff4e,"[{'system': 'GDC', 'value': '9148502d-cb91-41c...",e1c60464-3f4f-400b-bbfa-0e96f826d901.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-MSK,drs://dg.4DFC:9148502d-cb91-41c6-b382-62639010...,3825,20d837abab0ffa04e06e5de5f35108e8,Genomic,None,None
4,7aef2592-bb88-44fe-886c-22e0e3fd4aac,"[{'system': 'GDC', 'value': '7aef2592-bb88-44f...",9010b2f6-5eb0-4c68-b42a-fb271d8f5ba2.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-MSK,drs://dg.4DFC:7aef2592-bb88-44fe-886c-22e0e3fd...,1051,3f36395c16a0fc18171d490f1039efcb,Genomic,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,d91dbc00-747a-4c78-9fe9-d3d599df2656,"[{'system': 'GDC', 'value': 'd91dbc00-747a-4c7...",41a6475c-09bf-4b0c-b568-a5ce5d8f7a23.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-GRCC,drs://dg.4DFC:d91dbc00-747a-4c78-9fe9-d3d599df...,3465,fa2269804bffbba348da639311efe254,Genomic,None,None
96,87b76b2b-7f27-49b1-8989-c1c3820e5661,"[{'system': 'GDC', 'value': '87b76b2b-7f27-49b...",39444fc1-0c79-44c6-a1dc-9f86e72835e1.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-GRCC,drs://dg.4DFC:87b76b2b-7f27-49b1-8989-c1c3820e...,2196,68f202798873dfdd85b5680cb6263388,Genomic,None,None
97,c609be50-3cc3-447a-acbc-4d82b4c30e72,"[{'system': 'GDC', 'value': 'c609be50-3cc3-447...",fa186f29-1bd9-4b82-9daf-d606a6ddef11.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-GRCC,drs://dg.4DFC:c609be50-3cc3-447a-acbc-4d82b4c3...,3565,b1b7e7322180b6764e27faac63e77bc2,Genomic,None,None
98,e47498b5-27ae-46f7-8ee5-cf310360c598,"[{'system': 'GDC', 'value': 'e47498b5-27ae-46f...",9a9009af-e4ce-4477-9af7-851c82e61802.genie.ali...,Simple Nucleotide Variation,Masked Annotated Somatic Mutation,MAF,GENIE-MDA,drs://dg.4DFC:e47498b5-27ae-46f7-8ee5-cf310360...,1052,919e44b2808e2de2281ba9275698ba1d,Genomic,None,None


## Example queries

Now that we can create a query with ```Q()``` function, let's see how we can combine multiple conditions.

There are three operators available:
* ```And()```
* ```Or()```
* ```From()```

The following examples show how those operators work in practice.

### Query 1

**Find data for subjects who were diagnosed after the age of 50 and who were investigated as part of the TCGA-OV project.**

In [5]:
q1 = Q('sex = "male"')
q2 = Q('identifier.system = "GDC"')
q3 = Q('identifier.system = "PDC"')

r2 = q1.And(q2)
r3 = q1.And(q3)

#taco = r2.run()
#taco

burrito = r3.run()


Getting results from database

Total execution time: 15289 ms


In [6]:
burrito


            QueryID: 234efbf5-418d-4d81-aac3-43a9c196ba37
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Subjects_v3_0_w_RS.id) as rn, all_Subjects_v3_0_w_RS.id AS id, all_Subjects_v3_0_w_RS.identifier AS identifier, all_Subjects_v3_0_w_RS.species AS species, all_Subjects_v3_0_w_RS.sex AS sex, all_Subjects_v3_0_w_RS.race AS race, all_Subjects_v3_0_w_RS.ethnicity AS ethnicity, all_Subjects_v3_0_w_RS.days_to_birth AS days_to_birth, all_Subjects_v3_0_w_RS.subject_associated_project AS subject_associated_project, all_Subjects_v3_0_w_RS.vital_status AS vital_status, all_Subjects_v3_0_w_RS.age_at_death AS age_at_death, all_Subjects_v3_0_w_RS.cause_of_death AS cause_of_death, all_Subjects_v3_0_w_RS.Files AS Files, all_Subjects_v3_0_w_RS.ResearchSubject AS ResearchSubject FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS LEFT JOIN UNNEST(all_Subjects_v3_0_w_RS.identifier) AS _identifier WHERE ((UPPER(all_Subjects_v3_0_w_RS.

In [7]:
q1 = Q('sex = "male"')

taco = q1.run()
taco

Getting results from database

Total execution time: 21690 ms



            QueryID: e5c8c1b2-def0-49ef-af4c-fe78c2236780
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Subjects_v3_0_w_RS.id) as rn, all_Subjects_v3_0_w_RS.id AS id, all_Subjects_v3_0_w_RS.identifier AS identifier, all_Subjects_v3_0_w_RS.species AS species, all_Subjects_v3_0_w_RS.sex AS sex, all_Subjects_v3_0_w_RS.race AS race, all_Subjects_v3_0_w_RS.ethnicity AS ethnicity, all_Subjects_v3_0_w_RS.days_to_birth AS days_to_birth, all_Subjects_v3_0_w_RS.subject_associated_project AS subject_associated_project, all_Subjects_v3_0_w_RS.vital_status AS vital_status, all_Subjects_v3_0_w_RS.age_at_death AS age_at_death, all_Subjects_v3_0_w_RS.cause_of_death AS cause_of_death, all_Subjects_v3_0_w_RS.Files AS Files, all_Subjects_v3_0_w_RS.ResearchSubject AS ResearchSubject FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS WHERE (UPPER(all_Subjects_v3_0_w_RS.sex) = UPPER('male'))) as results WHERE rn = 1
            Offset: 0

In [8]:
from pandas import DataFrame,concat
q = query('ResearchSubject.primary_diagnosis_condition LIKE "Lung%" AND sex = "male"').run()
print(q)
df = DataFrame()
for i in q.paginator(to_df=True):
    print(len(i))
    df = concat([df,i])

Getting results from database

Total execution time: 9645 ms

            QueryID: 81881ecc-474a-410a-894a-33263a3b3afe
            Query:SELECT results.* EXCEPT(rn) FROM (SELECT ROW_NUMBER() OVER (PARTITION BY all_Subjects_v3_0_w_RS.id) as rn, all_Subjects_v3_0_w_RS.id AS id, all_Subjects_v3_0_w_RS.identifier AS identifier, all_Subjects_v3_0_w_RS.species AS species, all_Subjects_v3_0_w_RS.sex AS sex, all_Subjects_v3_0_w_RS.race AS race, all_Subjects_v3_0_w_RS.ethnicity AS ethnicity, all_Subjects_v3_0_w_RS.days_to_birth AS days_to_birth, all_Subjects_v3_0_w_RS.subject_associated_project AS subject_associated_project, all_Subjects_v3_0_w_RS.vital_status AS vital_status, all_Subjects_v3_0_w_RS.age_at_death AS age_at_death, all_Subjects_v3_0_w_RS.cause_of_death AS cause_of_death, all_Subjects_v3_0_w_RS.Files AS Files, all_Subjects_v3_0_w_RS.ResearchSubject AS ResearchSubject FROM gdc-bq-sample.dev.all_Subjects_v3_0_w_RS AS all_Subjects_v3_0_w_RS LEFT JOIN UNNEST(all_Subjects_v3_0_w_RS.ResearchSubject) AS _ResearchSubje

In [9]:
df

,id,identifier,species,sex,race,ethnicity,days_to_birth,subject_associated_project,vital_status,age_at_death,cause_of_death,Files,ResearchSubject
0,P064,"[{'system': 'PDC', 'value': 'P064'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[d996e6de-4485-11ea-bf04-0ef15c86e253, 75ee36b...",[{'id': '24b3ab9d-8beb-11ea-b1fd-0aad30af8a83'...
1,P068,"[{'system': 'PDC', 'value': 'P068'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[0da8630a-e8ca-4f20-acd5-6fc91ec27d65, bc2b104...",[{'id': '24b3b0e7-8beb-11ea-b1fd-0aad30af8a83'...
2,P071,"[{'system': 'PDC', 'value': 'P071'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[2f87638f-4c9f-43ce-935b-df01e9d5161c, eaefecf...",[{'id': '24b3b47b-8beb-11ea-b1fd-0aad30af8a83'...
3,P038,"[{'system': 'PDC', 'value': 'P038'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[23527043-4fdb-4cc4-81b7-2b6d46c490b7, c563492...",[{'id': '24b387fe-8beb-11ea-b1fd-0aad30af8a83'...
4,P015,"[{'system': 'PDC', 'value': 'P015'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[10bf0fb4-4495-11ea-bf04-0ef15c86e253, 257d9e8...",[{'id': '24b368d8-8beb-11ea-b1fd-0aad30af8a83'...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,P005,"[{'system': 'PDC', 'value': 'P005'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[7ed2372d-3bcd-11ea-bf04-0ef15c86e253, 7e1f4b5...",[{'id': '24b35cd4-8beb-11ea-b1fd-0aad30af8a83'...
3,P084,"[{'system': 'PDC', 'value': 'P084'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[18de756b-4497-11ea-bf04-0ef15c86e253, 2cf854f...",[{'id': '24b3c882-8beb-11ea-b1fd-0aad30af8a83'...
4,P047,"[{'system': 'PDC', 'value': 'P047'}]",Homo sapiens,male,not reported,not reported,None,[Academia Sinica LUAD-100],None,None,None,"[99cb0585-cbc6-4ed8-96dc-c2f8ec16fde2, 94d3918...",[{'id': '24b39432-8beb-11ea-b1fd-0aad30af8a83'...
5,C3L-02624,"[{'system': 'PDC', 'value': 'C3L-02624'}, {'sy...",Homo sapiens,male,not reported,not reported,-23126,"[CPTAC3-Discovery, cptac_lscc]",Dead,832,Cancer Related,"[da89d0ec-5af4-48fa-90ee-98279f34c5ff, 2bac942...",[{'id': '1c63ed9b-d028-11ea-b1fd-0aad30af8a83'...


In [10]:
r.to_dataframe()

NameError: name 'r' is not defined

### Query 2

**Find data for donors with melanoma (Nevi and Melanomas) diagnosis and who were diagnosed before the age of 30.**

In [ ]:
q1 = Q('ResearchSubject.Specimen.primary_disease_type = "Nevi and Melanomas"')
q2 = Q('ResearchSubject.Diagnosis.age_at_diagnosis < 30*365')

q = q1.And(q2)
r = q.run()

print(r)

In addition, we can check how many records come from particular systems by adding one more condition to the query:

In [ ]:
q1 = Q('ResearchSubject.Specimen.primary_disease_type = "Nevi and Melanomas"')
q2 = Q('ResearchSubject.Diagnosis.age_at_diagnosis < 30*365')
q3 = Q('ResearchSubject.Specimen.identifier.system = "GDC"')

q = q1.And(q2.And(q3))
r = q.run()

print(r)

By comparing the ```Count``` value of the two results we can see that all the patients returned in the initial query are coming from the GDC.

To explore the results further, we can fetch the patient JSON objects by iterating through the results:

In [ ]:
projects = set()

for patient in r:
    research_subjects = patient['ResearchSubject']
    for rs in research_subjects:
        projects.add(rs['member_of_research_project'])

print(projects)

The output shows the projects where _Nevi and Melanomas_ cases appear.

### Query 3

**Identify all samples that meet the following conditions:**

* **Sample is from primary tumor**
* **Disease is ovarian or breast cancer**
* **Subjects are females under the age of 60 years**

In [ ]:
tumor_type = Q('ResearchSubject.Specimen.source_material_type = "Primary Tumor"')
disease1 = Q('ResearchSubject.primary_diagnosis_site = "Ovary"')
disease2 = Q('ResearchSubject.primary_diagnosis_site = "Breast"')
demographics1 = Q('sex = "female"')
demographics2 = Q('days_to_birth > -60*365') # note that days_to_birth is a negative value

q1 = tumor_type.And(demographics1.And(demographics2))
q2 = disease1.Or(disease2)
q = q1.And(q2)

r = q.run()
print(r)

In this case, we have a result that contains more than 1000 records which is the default page size. To load the next 1000 records, we can use the ```next_page()``` method:

In [ ]:
r2 = r.next_page()

In [ ]:
print(r2)

Alternatively, we can use the ```offset``` argument to specify the record to start from:

```
...
r = q.run(offset=1000)
print(r)
```

### Query 4

**Find data for donors with "Ovarian Serous Cystadenocarcinoma" with proteomic and genomic data.**

**Note that disease type value denoting the same disease groups can be completely different within different systems. This is where CDA features come into play.** We first start by exploring the values available for this particular field in both systems.

In [ ]:
unique_terms('ResearchSubject.primary_diagnosis_condition', system="GDC")

Since “Ovarian Serous Cystadenocarcinoma” doesn’t appear in GDC values we decide to look into the PDC:

In [ ]:
unique_terms('ResearchSubject.primary_diagnosis_condition', system="PDC")

After examining the output, we see that it does come from the PDC. Hence, if we could first identify the data that has research subjects found within the PDC that have this particular disease type, and then further narrow down the results to include only the portion of the data that is present in GDC, we could get the records that we are looking for.

In [ ]:
q1 = Q('ResearchSubject.primary_diagnosis_condition = "Ovarian Serous Cystadenocarcinoma"')
q2 = Q('ResearchSubject.identifier.system = "PDC"')
q3 = Q('ResearchSubject.identifier.system = "GDC"')

q = q3.From(q1.And(q2))
r = q.run()

print(r)

As you can see, this is achieved by utilizing ```From``` operator. The ```From``` operator allows us to create queries from results of other queries. This is particularly useful when working with conditions that involve a single field which can take multiple different values for different items in a list that is being part of, e.g. we need ```ResearchSubject.identifier.system``` to be both “PDC” and “GDC” for a single patient. In such cases, ```And``` operator can’t help because it will return those entries where the field takes both values, which is zero entries.

In [ ]:
for i in Q.sql("SELECT * FROM `gdc-bq-sample.cda_mvp.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS` WHERE table_name = 'v3' Limit 5"):
    print(i)

In [ ]:
q1 = query('ResearchSubject.identifier.system = "GDC" FROM ResearchSubject.primary_diagnosis_condition = "Ovarian Serous Cystadenocarcinoma" AND ResearchSubject.identifier.system = "PDC"')
result = q1.run(async_call=True)
print(result)


## Data extraction and release information

In [ ]:
# If you are interested in the extraction dates or data release versions of GDC, PDC, or IDC that is in a table or view, execute this code

for i in Q.sql("SELECT option_value FROM `gdc-bq-sample.integration.INFORMATION_SCHEMA.TABLE_OPTIONS` WHERE table_name = 'all_v1'"):
    print(i)